In [166]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
import numpy as np
from keras.models import load_model
%matplotlib inline

In [167]:
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB()),
])

model = load_model('data/models/PooledGRU-GloVe.h5')

In [168]:
df = pd.read_csv('data/train.csv')

In [169]:
X = df['comment_text']
y = df['toxic']

In [170]:
text_clf.fit(X,y)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [171]:
text_clf.predict(['you suck and I fucking hate you'])

array([1])

In [172]:
text_clf.predict_proba(['you suck and I fucking hate you'])

array([[0.03838288, 0.96161712]])

In [173]:
text_clf.predict_proba(['that was great, thank you!'])

array([[0.99748885, 0.00251115]])

In [174]:
text_clf.predict(['that was great, thank you!'])

array([0])

Test on sample twitter dataset

In [175]:
from keras.preprocessing import text, sequence
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

X_train = train["comment_text"].fillna("fillna").values
X_test = test["comment_text"].fillna("fillna").values

# Takes about 40 sec
max_features = 30000
maxlen = 100
embed_size = 300
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))

In [176]:
twitter_df = pd.read_csv('data/sample_with_label.csv')
twitter_text = twitter_df["TweetText"].fillna("fillna").values
twitter_y = twitter_df["WasDeleted"].fillna("fillna").values
twitter_tokenized = tokenizer.texts_to_sequences(twitter_text)
twitter_tokenized = sequence.pad_sequences(twitter_tokenized, maxlen=maxlen)

In [177]:
twitter_df = pd.read_csv('data/sample_with_label.csv')
twitter_df.columns = ['Index','TweetID','ScreenName','TweetText','TweetDateTime','Followers','UserID','WasDeleted']
twitter_df.head()


,Index,TweetID,ScreenName,TweetText,TweetDateTime,Followers,UserID,WasDeleted
0,0,1244721273793646594,ShannePanne,b'Rhoa',2020-03-30 20:20:52,287,990004076829200385,0
1,1,1244721275232301058,25_ShadesOfK,b'@HisTemp_TAYtion Lol I learned from that shi...,2020-03-30 20:20:52,1189,624244930,0
2,2,1244721275936878593,PlagueJesterSky,b'Trying to set up the stream and I keep getti...,2020-03-30 20:20:52,415,755613447702847488,0
3,3,1244721278650650624,spitbull1963,b'@EricksonReal @Ilhan And you can test as muc...,2020-03-30 20:20:53,16,47425986,0
4,4,1244721278931664896,kamanfrancis,b'Me taking notes for the future when I\xe2\x8...,2020-03-30 20:20:53,1928,269295980,0


In [178]:
from sklearn.model_selection import train_test_split
tw_train, tw_test, twy_train, twy_test = train_test_split(twitter_tokenized, twitter_y, test_size=0.25, random_state=42)
tw_train_df, tw_test_df, twy_train, twy_test = train_test_split(twitter_df, twitter_y, test_size=0.25, random_state=42)

y_dummy = np.zeros((len(twy_train),6))
y_dummy[:,0] = twy_train
y_dummy[:,1] = twy_train
y_dummy[:,2] = twy_train
y_dummy[:,3] = twy_train
y_dummy[:,4] = twy_train
y_dummy[:,5] = twy_train

In [179]:
hist = model.fit(tw_train, y_dummy, batch_size=32, epochs=2, validation_split=0.15 ,verbose=1)

Train on 18773 samples, validate on 3313 samples
Epoch 1/2
18773/18773 [==============================] - 119s 6ms/step - loss: 0.3089 - accuracy: 0.9083 - val_loss: 0.3157 - val_accuracy: 0.9013
Epoch 2/2
18773/18773 [==============================] - 112s 6ms/step - loss: 0.2726 - accuracy: 0.9105 - val_loss: 0.3403 - val_accuracy: 0.9029


In [180]:
tw_test_df['ToxicityProba'] = text_clf.predict_proba(tw_test_text['TweetText'])[:,1]
tw_test_df['ToxicityProba2'] = model.predict(tw_test, verbose=1)[:,1]

/home/martin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


7363/7363 [==============================] - 5s 681us/step


/home/martin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [181]:
tw_test_df

,Index,TweetID,ScreenName,TweetText,TweetDateTime,Followers,UserID,WasDeleted,ToxicityProba,ToxicityProba2
7467,7467,1244726700241489921,CUILLINNN,b'@MrBarrymore Barrynomore do you still need t...,2020-03-30 20:42:25,4,214188916,0,0.002716,0.041373
14494,14494,1244731348843184128,aobamoca,b'can\xe2\x80\x99t believe i made a friend ove...,2020-03-30 21:00:54,131,1080311886674124800,0,0.043622,0.024868
10157,10157,1244728506896060418,crespo_08,b'@ardillacuqui Yo por hoy ya he tenido sufici...,2020-03-30 20:49:36,196,3747451583,0,0.326248,0.014255
16262,16262,1244732433297215491,anfuaruuu,"b""just scrolled through some ppl's pages and i...",2020-03-30 21:05:12,72,101923117,0,0.169993,0.173239
16367,16367,1244732492768313345,JstMeMyslfNTy,"b""I hit you up almost everyday to ask you how ...",2020-03-30 21:05:26,5,1159401755190411266,0,0.458037,0.048678
...,...,...,...,...,...,...,...,...,...,...
1574,1574,1244722872779780097,0DDYNUFF_,b'all three but :((',2020-03-30 20:27:13,298,1216882530336550912,0,0.058502,0.010431
6815,6815,1244726284845973505,OsakaSan214,b'@EricHolthaus Main bread-earner in my family...,2020-03-30 20:40:46,25,2669126794,0,0.094359,0.018407
1349,1349,1244722703711567873,Go_Los,b'Why would you complain about dinner being co...,2020-03-30 20:26:33,869,70854688,0,0.004376,0.035078
27363,27363,1244738871549726721,BrummieBoy5,"b'At the end of my bread bought last week, las...",2020-03-30 21:30:47,147,1196065943467810816,0,0.018272,0.015493


In [182]:
tw_test_df = tw_test_df.sort_values(by=['ToxicityProba2'],ascending=False)
tw_test_df.head(10)


,Index,TweetID,ScreenName,TweetText,TweetDateTime,Followers,UserID,WasDeleted,ToxicityProba,ToxicityProba2
14370,14370,1244731284062167043,userentry,b'@PacdWeu Son of a bitch is Chinese... \n\nF...,2020-03-30 21:00:38,244,2296839540,1,0.025044,0.770924
6025,6025,1244725754148958216,renee_aziz,b'Retweet if you put me on my knees and fuck m...,2020-03-30 20:38:40,7,1244351220283736072,1,0.056965,0.721708
3222,3222,1244723937193242625,Shannon31436914,b'Retweet if you put me on my knees and fuck m...,2020-03-30 20:31:27,3,1244648438677024768,1,0.032908,0.721708
22485,22485,1244735968764026880,Candice96394784,b'Retweet if you put me on my knees and fuck m...,2020-03-30 21:19:15,0,1244716638559223810,1,0.095844,0.712123
8394,8394,1244727341282136067,userentry,b'@Mike_IMC Son of a bitch is Chinese... Fuck ...,2020-03-30 20:44:58,244,2296839540,1,0.054617,0.654580
12042,12042,1244729776927113224,aribigd,b'if you\xe2\x80\x99re bored and horny and wan...,2020-03-30 20:54:39,158,1059172170201513984,1,0.045999,0.636272
18776,18776,1244733812422127617,aribigd,b'if you\xe2\x80\x99re bored and horny and wan...,2020-03-30 21:10:41,159,1059172170201513984,1,0.029701,0.636272
5621,5621,1244725479598370821,aribigd,b'if you\xe2\x80\x99re bored and horny and wan...,2020-03-30 20:37:34,158,1059172170201513984,1,0.014327,0.636272
15609,15609,1244732039766708229,aribigd,b'if you\xe2\x80\x99re bored and horny and wan...,2020-03-30 21:03:38,158,1059172170201513984,1,0.095844,0.636272
2055,2055,1244723164749410304,RachelP50019129,b'Retweet if you want to fuck me in a tight as...,2020-03-30 20:28:22,0,1244677473293524999,0,0.202176,0.615601


In [183]:
def is_response_tweet(tweet):
    return '@' in tweet

In [184]:
tw_test_df['IsResponseTweet'] = tw_test_df['TweetText'].apply(is_response_tweet)

In [185]:
response_tweets_df = tw_test_df[tw_test_df['IsResponseTweet']]
print(response_tweets_df.shape)
print(tw_test_df.shape)
print(tw_test_df.shape)

(2909, 11)
(7363, 11)
(7363, 11)


In the top 50 predicted toxic response tweets, 10% of them were deleted.

In [186]:
response_tweets_df.head(50)['WasDeleted'].values.sum()/50

0.32

By contrast, in the bottom 50 predicted toxic response tweets, 2% of them were deleted.

In [187]:
response_tweets_df.tail(50)['WasDeleted'].values.sum()/50

0.06

There are a lot of tweets with pornographic content that are getting deleted...

In [188]:
tw_test_df[tw_test_df['WasDeleted']==1].head(5)

,Index,TweetID,ScreenName,TweetText,TweetDateTime,Followers,UserID,WasDeleted,ToxicityProba,ToxicityProba2,IsResponseTweet
14370,14370,1244731284062167043,userentry,b'@PacdWeu Son of a bitch is Chinese... \n\nF...,2020-03-30 21:00:38,244,2296839540,1,0.025044,0.770924,True
6025,6025,1244725754148958216,renee_aziz,b'Retweet if you put me on my knees and fuck m...,2020-03-30 20:38:40,7,1244351220283736072,1,0.056965,0.721708,False
3222,3222,1244723937193242625,Shannon31436914,b'Retweet if you put me on my knees and fuck m...,2020-03-30 20:31:27,3,1244648438677024768,1,0.032908,0.721708,False
22485,22485,1244735968764026880,Candice96394784,b'Retweet if you put me on my knees and fuck m...,2020-03-30 21:19:15,0,1244716638559223810,1,0.095844,0.712123,False
8394,8394,1244727341282136067,userentry,b'@Mike_IMC Son of a bitch is Chinese... Fuck ...,2020-03-30 20:44:58,244,2296839540,1,0.054617,0.654580,True
